In [1]:
import json
import numpy as np
import faiss
from openai import OpenAI  # Upstage API 사용

client = OpenAI(
    api_key="up_3a0n6x50BZ1Fy057c9RXOVlzIMUEe",
    base_url="https://api.upstage.ai/v1/solar"
)

In [2]:
def normalize(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / norms

In [6]:
# FAISS 인덱스와 chunk-mapping 파일을 다시 로드

res = faiss.StandardGpuResources()
index = faiss.read_index("knn_index_cosine.faiss")

gpu_index = faiss.index_cpu_to_gpu(res, 0, index)

with open("chunk_mappings.json", "r") as f:
    chunk_doc_mapping = json.load(f)

# 쿼리 입력 후 Upstage API를 통해 임베딩 생성
query = "나무의 분류에 대해 조사해 보기 위한 방법은?"
query_result = client.embeddings.create(
    model="solar-embedding-1-large-passage",
    input=query
).data[0].embedding

query_embedding = np.array(query_result).reshape(1, -1)

# 쿼리 임베딩 정규화
normalized_query = normalize(query_embedding)

# KNN에서 가장 유사한 5개의 chunk 검색
k = 5
distances, indices = gpu_index.search(normalized_query, k)

# 검색된 chunk 정보 확인 및 원본 문서 내용과 연결
for idx in indices[0]:
    chunk_info = chunk_doc_mapping[idx]
    print(f"검색된 Chunk ID: {chunk_info['chunk_id']}")
    print(f"연관된 문서 ID: {chunk_info['doc_id']}")
    print(f"Chunk 내용: {chunk_info['content']}")
    print("-" * 50)

AttributeError: module 'faiss' has no attribute 'StandardGpuResources'